In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from matplotlib.gridspec import GridSpec
from scipy.special import boxcox1p
import warnings

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

import pandas as pd 
import numpy as np 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.preprocessing import LabelEncoder


from sklearn.compose import make_column_transformer
from itertools import combinations 


warnings.filterwarnings("ignore") # ignoring annoying warnings

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1) 
pd.set_option("display.max_rows", 500)

In [2]:
# READ THE WHOLE DATASET:
# We load and read the whole dataset (train + test) because we need to clean and preprocess all of it!!!!
#All the procedures must be applied to the entire dataset before split it!

#Step 1: Load train and test sets

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

#Target
target = "Loan_Status"
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
#Save the 'Id' column
train_ID = train['Loan_ID']
test_ID = test['Loan_ID']

#Drop ID from both datasets
# train.drop("Id", axis = 1, inplace = True)
# test.drop("Id", axis = 1, inplace = True)

# Shape of the datasets
train.shape, test.shape, train.shape[0], test.shape[0]

#Separar y convertir el target en DataFrame
y_train = train[target].to_frame()

#Step 2: Concatenate both datasets into one

df = pd.concat((train,test), sort = False).reset_index(drop = True)

#drop target and id columns
df.drop(columns = [target, "Loan_ID"], axis = 1, inplace = True)
train.drop(columns = ["Loan_ID"], axis = 1, inplace = True)
test.drop(columns = ["Loan_ID"], axis = 1, inplace = True)

In [5]:
df.shape,train.shape, test.shape, train.shape[0], test.shape[0]

((981, 11), (614, 12), (367, 11), 614, 367)

In [6]:
le = LabelEncoder()
train["Loan_Status"] = pd.Series(le.fit_transform(train[target]))

In [7]:
def summary_2(df, target):
    num = df.select_dtypes(exclude= object).columns
    cols = ["Dtype","Uniques", "Nulls","% Nulls", "Skew", "Kurtosis", "Correlation"]
    ind = df.columns
    dict = {}
    dict["Dtype"] = [df[i].dtype for i in ind]
    dict["Uniques"] = [df[i].nunique() for i in ind]
    dict["Nulls"] =[df[i].isnull().sum().sum() for i in ind]
    dict["% Nulls"] = np.round([df[i].isnull().sum().sum()/len(df[i]) for i in ind], decimals = 2)
    dict["Skew"] =  np.round(df.skew(), decimals = 3) 
    dict["Kurtosis"] = np.round(df.kurt(), decimals = 3)
    dict["Correlation"] = np.round(df.corr()[target], decimals = 3)
    summary = pd.DataFrame(dict, columns = cols, index = ind)
    return summary

sum = summary_2(train, target).sort_values(by = "Dtype").sort_values(by = "Correlation", ascending = False)
sum

,Dtype,Uniques,Nulls,% Nulls,Skew,Kurtosis,Correlation
Loan_Status,int32,2,0,0.00,-0.810,-1.348,1.000
Credit_History,float64,2,50,0.08,-1.882,1.549,0.562
ApplicantIncome,int64,505,0,0.00,6.540,60.541,-0.005
Loan_Amount_Term,float64,10,14,0.02,-2.362,6.673,-0.021
LoanAmount,float64,203,22,0.04,2.678,10.402,-0.037
CoapplicantIncome,float64,287,0,0.00,7.492,84.956,-0.059
Gender,object,2,13,0.02,NaN,NaN,NaN
Married,object,2,3,0.00,NaN,NaN,NaN
Dependents,object,4,15,0.02,NaN,NaN,NaN
Education,object,2,0,0.00,NaN,NaN,NaN


In [9]:
train.shape

(614, 12)

In [10]:
#Nulls in train and test
#Train
train.isnull().sum()

Gender               13
Married              3 
Dependents           15
Education            0 
Self_Employed        32
ApplicantIncome      0 
CoapplicantIncome    0 
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area        0 
Loan_Status          0 
dtype: int64

In [12]:
train.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [ ]:
df = pd.get_dummies(df, drop_first = True )

In [ ]:
df.head(2)

In [ ]:
test.isnull().sum()